In [1]:

import gc
import time

import lightgbm as lgb
import numpy as np
import pandas as pd
from sklearn import metrics
import matplotlib.pyplot as plt

%matplotlib inline

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
pd.set_option('max_rows', None)
import os
path = ''
# Any results you write to the current directory are saved as output.

In [2]:
DATA_PATH = "datasets"
csv_path = os.path.join(DATA_PATH, "atec_anti_fraud_train.csv")
# 读取训练集和测试集
train = pd.read_csv(csv_path)
#train = pd.read_csv(path + 'atec_anti_fraud_train.csv')
# train = train[train.label.isin([0, 1])]
train[train['label'] == -1]['label'] = 0

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [3]:
train.shape

(994731, 300)

In [4]:
csv_path = os.path.join(DATA_PATH, "atec_anti_fraud_test_a.csv")
test = pd.read_csv(csv_path)
#test = pd.read_csv(path + 'atec_anti_fraud_test_a.csv')

In [5]:
# 合并train与test，统一处理
test['label'] = -1
data = pd.concat((train, test), axis=0)

In [6]:
del train, test
gc.collect()

21

In [9]:
pd.set_option('display.max_columns',2000)
data.head()

,date,f1,f10,f100,f101,f102,f103,f104,f105,f106,f107,f108,f109,f11,f110,f111,f112,f113,f114,f115,f116,f117,f118,f119,f12,f120,f121,f122,f123,f124,f125,f126,f127,f128,f129,f13,f130,f131,f132,f133,f134,f135,f136,f137,f138,f139,f14,f140,f141,f142,f143,f144,f145,f146,f147,f148,f149,f15,f150,f151,f152,f153,f154,f155,f156,f157,f158,f159,f16,f160,f161,f162,f163,f164,f165,f166,f167,f168,f169,f17,f170,f171,f172,f173,f174,f175,f176,f177,f178,f179,f18,f180,f181,f182,f183,f184,f185,f186,f187,f188,f189,f19,f190,f191,f192,f193,f194,f195,f196,f197,f198,f199,f2,f20,f200,f201,f202,f203,f204,f205,f206,f207,f208,f209,f21,f210,f211,f212,f213,f214,f215,f216,f217,f218,f219,f22,f220,f221,f222,f223,f224,f225,f226,f227,f228,f229,f23,f230,f231,f232,f233,f234,f235,f236,f237,f238,f239,f24,f240,f241,f242,f243,f244,f245,f246,f247,f248,f249,f25,f250,f251,f252,f253,f254,f255,f256,f257,f258,f259,f26,f260,f261,f262,f263,f264,f265,f266,f267,f268,f269,f27,f270,f271,f272,f273,f274,f275,f276,f277,f278,f279,f28,f280,f281,f282,f283,f284,f285,f286,f287,f288,f289,f29,f290,f291,f292,f293,f294,f295,f296,f297,f3,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f4,f40,f41,f42,f43,f44,f45,f46,f47,f48,f49,f5,f50,f51,f52,f53,f54,f55,f56,f57,f58,f59,f6,f60,f61,f62,f63,f64,f65,f66,f67,f68,f69,f7,f70,f71,f72,f73,f74,f75,f76,f77,f78,f79,f8,f80,f81,f82,f83,f84,f85,f86,f87,f88,f89,f9,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99,id,label,year,month,day
0,20171103,0,1,15.0,30.0,4.0,14.0,34.0,70.0,142.0,3.0,15.0,41.0,1,79.0,191.0,192.0,190.0,190.0,190.0,190.0,191.0,192.0,191.0,2,190.0,190.0,190.0,32.0,33.0,25.0,25.0,24.0,25.0,31.0,0,79.0,42.0,42.0,42.0,68.0,32.0,32.0,28.0,27.0,28.0,2,27.0,32.0,235.0,235.0,126.0,126.0,127.0,126.0,235.0,192.0,0,191.0,190.0,189.0,189.0,301.0,2.0,1.0,1.0,1.0,1.0,2,2.0,2.0,3.0,3.0,4.0,1.0,2.0,2.0,1.0,2.0,2,1.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,2.0,1.0,2,1.0,1.0,2.0,4.0,3.0,4.0,2.0,1.0,1.0,1.0,1,2.0,1.0,2.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,0,31.0,2.0,1.0,1.0,2.0,3.0,3.0,3.0,14.0,41.0,80.0,61.0,167.0,2.0,2.0,4.0,3.0,3.0,4.0,6.0,6.0,2.0,142.0,2.0,4.0,3.0,2.0,2.0,3.0,4.0,3.0,3.0,4.0,245.0,1.0,3.0,3.0,4.0,2.0,2.0,2.0,2.0,8.0,2.0,0.0,2.0,3.0,3.0,3.0,2.0,5.0,5.0,10.0,16.0,2.0,1.0,2.0,4.0,5.0,7.0,2.0,2.0,2.0,2.0,1.0,2.0,0.0,2.0,2.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,27.0,297.0,0.0,302.0,384.0,770.0,20.0,99.0,102.0,124.0,140.0,301.0,312.0,0.0,328.0,85.0,302.0,201.0,203.0,203.0,61.0,201.0,0,1.0,0.0,0.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,100807.0,1.0,0.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,2.0,0,1.0,2.0,2.0,3.0,96.0,38.0,301.0,302.0,302.0,302.0,5,301.0,302.0,40.0,79.0,40.0,79.0,2.0,3.0,4.0,3.0,1,41.0,80.0,96.5,721.1,3252.0,154.5,394.5,2.0,1.0,2.0,1,1.0,2.0,1.0,2.0,1.0,1.0,2.0,1.0,4.0,8.0,f10eb20f31cf7063ee8bdbd1272214e4d7e0193c8dbce4...,0,2017,11,3
1,20170917,0,1,11.0,12.0,4.0,12.0,23.0,26.0,27.0,5.0,12.0,24.0,0,27.0,122.0,121.0,16.0,121.0,121.0,122.0,121.0,122.0,121.0,2,16.0,121.0,122.0,32.0,31.0,13.0,27.0,26.0,27.0,32.0,1,79.0,40.0,39.0,39.0,63.0,31.0,31.0,15.0,28.0,29.0,2,29.0,31.0,231.0,232.0,17.0,99.0,99.0,98.0,232.0,121.0,2,122.0,17.0,121.0,121.0,302.0,2.0,1.0,1.0,2.0,2.0,1,2.0,2.0,2.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,1,1.0,2.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,2.0,2,1.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,1,14.0,2.0,1.0,2.0,1.0,1.0,4.0,5.0,12.0,24.0,26.0,13.0,27.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,2.0,48.0,1.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,48.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,1.0,2.0,1.0,0.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,2.0,2.0,2.0,0.0,2.0,2.0,1.0,1.0,2.0,1.0,2.0,2.0,2.0,1.0,0.0,2.0,2.0,1.0,3.0,2.0,2.0,2.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,13.0,158.0,1.0,301.0,335.0,398.0,4.0,54.0,109.0,112.0,113.0,302.0,324.0,1.0,391.0,13.0,302.0,160.0,160.0,161.0,8.0,160.0,1,0.0,0.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,100805.0,0.0,1.0,0.0,1.0,1.0,2.0,4.0,4.0,5.0,1.0,1,1.0,4.0,3.0,3.0,79.0,12.0,302.0,32.0,301.0,302.0,5,302.0,302.0,23.0,26.0,23.0,26.0,2.0,2.0,3

In [8]:
# 处理date，年、月、日
data['year'] = data['date'].apply(lambda x: int(str(x)[0:4]))
data['month'] = data['date'].apply(lambda x: int(str(x)[4:6]))
data['day'] = data['date'].apply(lambda x: int(str(x)[6:8]))

In [10]:
# 得到int，float的列名
float_cols = data.blocks['float64'].columns
int_cols = data.blocks['int64'].columns

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:3884: FutureWarning: as_blocks is deprecated and will be removed in a future version
  return self.as_blocks()


In [21]:
print(float_cols.shape)
print(int_cols.shape)

(279,)
(23,)


In [18]:
d1 = dict((col, -1) for col in int_cols)
d2 = dict((col, -1.) for col in float_cols)

In [19]:
print(d1)
print(d2)

{'date': -1, 'f1': -1, 'f10': -1, 'f11': -1, 'f12': -1, 'f13': -1, 'f14': -1, 'f15': -1, 'f16': -1, 'f17': -1, 'f18': -1, 'f19': -1, 'f2': -1, 'f3': -1, 'f4': -1, 'f6': -1, 'f7': -1, 'f8': -1, 'f9': -1, 'label': -1, 'year': -1, 'month': -1, 'day': -1}
{'f100': -1.0, 'f101': -1.0, 'f102': -1.0, 'f103': -1.0, 'f104': -1.0, 'f105': -1.0, 'f106': -1.0, 'f107': -1.0, 'f108': -1.0, 'f109': -1.0, 'f110': -1.0, 'f111': -1.0, 'f112': -1.0, 'f113': -1.0, 'f114': -1.0, 'f115': -1.0, 'f116': -1.0, 'f117': -1.0, 'f118': -1.0, 'f119': -1.0, 'f120': -1.0, 'f121': -1.0, 'f122': -1.0, 'f123': -1.0, 'f124': -1.0, 'f125': -1.0, 'f126': -1.0, 'f127': -1.0, 'f128': -1.0, 'f129': -1.0, 'f130': -1.0, 'f131': -1.0, 'f132': -1.0, 'f133': -1.0, 'f134': -1.0, 'f135': -1.0, 'f136': -1.0, 'f137': -1.0, 'f138': -1.0, 'f139': -1.0, 'f140': -1.0, 'f141': -1.0, 'f142': -1.0, 'f143': -1.0, 'f144': -1.0, 'f145': -1.0, 'f146': -1.0, 'f147': -1.0, 'f148': -1.0, 'f149': -1.0, 'f150': -1.0, 'f151': -1.0, 'f152': -1.0, 'f153

In [20]:
dict(d1, **d2)

{'date': -1,
 'day': -1,
 'f1': -1,
 'f10': -1,
 'f100': -1.0,
 'f101': -1.0,
 'f102': -1.0,
 'f103': -1.0,
 'f104': -1.0,
 'f105': -1.0,
 'f106': -1.0,
 'f107': -1.0,
 'f108': -1.0,
 'f109': -1.0,
 'f11': -1,
 'f110': -1.0,
 'f111': -1.0,
 'f112': -1.0,
 'f113': -1.0,
 'f114': -1.0,
 'f115': -1.0,
 'f116': -1.0,
 'f117': -1.0,
 'f118': -1.0,
 'f119': -1.0,
 'f12': -1,
 'f120': -1.0,
 'f121': -1.0,
 'f122': -1.0,
 'f123': -1.0,
 'f124': -1.0,
 'f125': -1.0,
 'f126': -1.0,
 'f127': -1.0,
 'f128': -1.0,
 'f129': -1.0,
 'f13': -1,
 'f130': -1.0,
 'f131': -1.0,
 'f132': -1.0,
 'f133': -1.0,
 'f134': -1.0,
 'f135': -1.0,
 'f136': -1.0,
 'f137': -1.0,
 'f138': -1.0,
 'f139': -1.0,
 'f14': -1,
 'f140': -1.0,
 'f141': -1.0,
 'f142': -1.0,
 'f143': -1.0,
 'f144': -1.0,
 'f145': -1.0,
 'f146': -1.0,
 'f147': -1.0,
 'f148': -1.0,
 'f149': -1.0,
 'f15': -1,
 'f150': -1.0,
 'f151': -1.0,
 'f152': -1.0,
 'f153': -1.0,
 'f154': -1.0,
 'f155': -1.0,
 'f156': -1.0,
 'f157': -1.0,
 'f158': -1.0,
 'f159'

In [22]:
# 处理缺失值
data = data.fillna(value=dict(d1, **d2))

In [23]:
train = data[data.label.isin([0, 1])]
test = data[data.label.isin([-1])]

In [24]:
del data
gc.collect()

2379

In [90]:
train.groupby('month').count()['date']

month
9     413043
10    494127
11     82836
Name: date, dtype: int64

In [91]:
gc.collect()

7

In [92]:
train = train.drop('id', axis=1)
res = test[['id']]
test = test.drop(['id', 'label'], axis=1)

val = train[train['month'] == 11]
train = train[train['month'] != 11]
train_label = train.pop('label')
val_label = val.pop('label')

In [93]:
def eval_func(y_pred, y_true):
    fpr, tpr, thresholds = metrics.roc_curve(y_pred, y_true, pos_label=1)
    score = 0.4 * tpr[np.argwhere(fpr < 0.001)[-1]] + 0.3 * tpr[np.argwhere(
        fpr < 0.005)[-1]] + 0.3 * tpr[np.argwhere(fpr < 0.01)[-1]]
    return 'score', score, True

In [94]:
def LGB_predict(train_x, train_y, val_x, val_y, test_x, res):
    print("LGB test")
#     clf = lgb.LGBMClassifier(
#         boosting_type='gbdt',
#         num_leaves=63,
#         reg_alpha=0.0,
#         reg_lambda=1,
#         max_depth=6,
#         n_estimators=1000,
#         objective='binary',
#         subsample=0.7,
#         colsample_bytree=0.7,
#         subsample_freq=1,
#         learning_rate=0.05,
#         min_child_weight=50,
#         min_data_in_leaf=5000,
#         random_state=2018,
#         verbose=0,
#         n_jobs=-1)
    clf = lgb.LGBMClassifier(
        boosting_type='gbdt',
        num_leaves=60,
        n_estimators=1000,
        objective='binary',
        learning_rate=0.05,
        feature_fraction=0.9,
        bagging_fraction=0.8,
        bagging_freq=5,
        random_state=2018,
        verbose=0,
        n_jobs=-1)

    clf.fit(train_x, train_y, eval_set=[(val_x, val_y), (train_x, train_y)], eval_metric=eval_func,early_stopping_rounds=50)
    res['score'] = clf.predict_proba(test_x)[:,1]
    # res['score'] = res['score'].apply(lambda x: float('%.6f' % x))
    res.to_csv('submission.csv', index=False)
    return clf

In [95]:
model = LGB_predict(train, train_label, val, val_label, test, res)

LGB test
[1]	training's binary_logloss: 0.645819	training's score: 0.482029	valid_0's binary_logloss: 0.645934	valid_0's score: 0.409399
Training until validation scores don't improve for 50 rounds.
[2]	training's binary_logloss: 0.602992	training's score: 0.488702	valid_0's binary_logloss: 0.603176	valid_0's score: 0.438469
[3]	training's binary_logloss: 0.564047	training's score: 0.491307	valid_0's binary_logloss: 0.564319	valid_0's score: 0.43343
[4]	training's binary_logloss: 0.528489	training's score: 0.50899	valid_0's binary_logloss: 0.528812	valid_0's score: 0.44157
[5]	training's binary_logloss: 0.495924	training's score: 0.515446	valid_0's binary_logloss: 0.496297	valid_0's score: 0.442733
[6]	training's binary_logloss: 0.46597	training's score: 0.53128	valid_0's binary_logloss: 0.466412	valid_0's score: 0.460465
[7]	training's binary_logloss: 0.438364	training's score: 0.534797	valid_0's binary_logloss: 0.438877	valid_0's score: 0.468992
[8]	training's binary_logloss: 0.41286

[64]	training's binary_logloss: 0.0410686	training's score: 0.618395	valid_0's binary_logloss: 0.0442501	valid_0's score: 0.523547
[65]	training's binary_logloss: 0.0401185	training's score: 0.619459	valid_0's binary_logloss: 0.04335	valid_0's score: 0.52093
[66]	training's binary_logloss: 0.0392021	training's score: 0.620352	valid_0's binary_logloss: 0.0424718	valid_0's score: 0.524031
[67]	training's binary_logloss: 0.0383303	training's score: 0.620072	valid_0's binary_logloss: 0.0416272	valid_0's score: 0.521705
[68]	training's binary_logloss: 0.0374981	training's score: 0.621118	valid_0's binary_logloss: 0.0408365	valid_0's score: 0.521609
[69]	training's binary_logloss: 0.0367011	training's score: 0.622507	valid_0's binary_logloss: 0.0401069	valid_0's score: 0.522771
[70]	training's binary_logloss: 0.0359442	training's score: 0.623399	valid_0's binary_logloss: 0.0393978	valid_0's score: 0.521705
[71]	training's binary_logloss: 0.0352291	training's score: 0.623877	valid_0's binary_

[127]	training's binary_logloss: 0.0204452	training's score: 0.665636	valid_0's binary_logloss: 0.0259823	valid_0's score: 0.5375
[128]	training's binary_logloss: 0.0203618	training's score: 0.666465	valid_0's binary_logloss: 0.0259353	valid_0's score: 0.538275
[129]	training's binary_logloss: 0.0202793	training's score: 0.667313	valid_0's binary_logloss: 0.0258975	valid_0's score: 0.537888
[130]	training's binary_logloss: 0.0202039	training's score: 0.667926	valid_0's binary_logloss: 0.0258701	valid_0's score: 0.537209
[131]	training's binary_logloss: 0.0201198	training's score: 0.669161	valid_0's binary_logloss: 0.0258283	valid_0's score: 0.537016
[132]	training's binary_logloss: 0.020051	training's score: 0.669477	valid_0's binary_logloss: 0.0257853	valid_0's score: 0.535562
[133]	training's binary_logloss: 0.0199756	training's score: 0.670848	valid_0's binary_logloss: 0.0257413	valid_0's score: 0.536337
[134]	training's binary_logloss: 0.0199073	training's score: 0.67101	valid_0's 

In [96]:
time.asctime( time.localtime(time.time())),model.get_params(), model.best_score_

('Mon Jun 11 22:56:14 2018',
 {'bagging_fraction': 0.8,
  'bagging_freq': 5,
  'boosting_type': 'gbdt',
  'class_weight': None,
  'colsample_bytree': 1.0,
  'feature_fraction': 0.9,
  'learning_rate': 0.05,
  'max_depth': -1,
  'min_child_samples': 20,
  'min_child_weight': 0.001,
  'min_split_gain': 0.0,
  'n_estimators': 1000,
  'n_jobs': -1,
  'num_leaves': 50,
  'objective': 'binary',
  'random_state': 2018,
  'reg_alpha': 0.0,
  'reg_lambda': 0.0,
  'silent': True,
  'subsample': 1.0,
  'subsample_for_bin': 200000,
  'subsample_freq': 1,
  'verbose': 0},
 defaultdict(dict,
             {'training': {'binary_logloss': 0.02036182831048949,
               'score': array([0.66646528])},
              'valid_0': {'binary_logloss': 0.02593530080810204,
               'score': array([0.53827519])}}))

In [50]:
time.asctime( time.localtime(time.time()) )

'Mon Jun 11 22:31:26 2018'